In [3]:
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [4]:
from data_helper import *

In [5]:
import warnings
warnings.filterwarnings("error")

In [65]:
def gradient2Image(W1):
    G = W1.reshape( 32, 32, 3, -1).transpose(3, 0, 1, 2)
    maxg = G.max()
    ming = G.min()
    if np.isinf(ming) or np.isinf(maxg):
        G = np.ones(G.shape)*154
    else:
        G = (G - ming)/(maxg-ming)*255
        
    
    return (G).astype(np.int).tolist()

def float_list2int(ll):
    ll = ll[np.logical_not(np.isnan(ll))]
    kk = ll[np.logical_not(np.isinf(ll))]

#     kk = (ll*1e6).astype(np.int)
#     kk[kk > 65088000000] = 65088000000
#     kk[kk < 0] = np.iinfo(np.int).max
#     print(kk[-10:])
    return kk.tolist()

In [66]:
import json
json.encoder.FLOAT_REPR =lambda x:format(x,'.3f')
    
class MyEncoder(json.JSONEncoder):
    def encode(self, obj):
        if isinstance(obj, float):
            return format(obj, '.2f')
        return json.JSONEncoder.encode(self, obj)
    
def save2json(hs, bs, lr, reg, num_epoch, val_acc, W1, stats):
    data = {
        "hs": hs,
        "bs": bs,
        "lr": lr,
        "reg": reg,
        "val_acc": val_acc,
        "num_epoch": num_epoch,
        "train_acc_history": float_list2int(stats["train_acc_history"]),
        "val_acc_history": float_list2int(stats["val_acc_history"]),
        
        "W1": gradient2Image(W1),
        "loss_history": float_list2int(stats["loss_history"]),
    }
#     if json_exist(hs, bs, lr, reg, num_epoch):
#         return
    with open(f'json/{hs}-{bs}-{lr}-{reg}-{num_epoch}.json', 'w') as f:
#         json.dump(data, f, indent=1)
        json.dump(data, f,separators=(',', ':'))

hs, bs, lr, reg, num_epoch, val_acc, W1, stats = get_pickle(3, 10, 0.004, 0, 8)

save2json(hs, bs, lr, reg, num_epoch, val_acc, W1, stats)

In [68]:
import warnings
warnings.filterwarnings("error")

hidden_size = [3, 5, 10, 50, 150]
# batch_size = [10, 50, 100, 200, 300, 400]
batch_size = [10, 50, 100, 200, 400]
# learning_rate = [4.5e-3, 4e-3, 3.5e-3, 3e-3, 2e-3, 1e-3, 5e-4, 1e-4]
# learning_rate = [4e-3, 3e-3, 1e-3, 5e-4, 1e-4]
learning_rate = [3e-3]
# regularization = [0, 0.1, 0.5, 1, 3, 5, 10]
regularization = [0, 0.1, 0.5, 1, 10]
num_epoch = 8

dtype = np.half
for lr in learning_rate:
    for bs in batch_size:
        for hs in hidden_size:
            for reg in regularization:
                if pickle_exist(hs, bs, lr, reg, num_epoch):
                    hs, bs, lr, reg, num_epoch, val_acc, W1, stats = get_pickle(hs, bs, lr, reg, num_epoch)
                    save2json(hs, bs, lr, reg, num_epoch, val_acc, W1, stats)
#                     print(f'For hs={hs}, bs={bs}, lr={lr}, reg={reg}, val_acc = {val_acc}')
                else:
                    print(f'For hs={hs}, bs={bs}, lr={lr}, reg={reg}, there is no data')

In [ ]:
# def save_general_pickle(hs, bs, lr, reg, num_epoch, val_acc, W1, stats):
#     W1 = W1.tolist()
#     for key in stats.keys():
#         stats[key] = stats[key].tolist()
#     obj = (hs, bs, lr, reg, num_epoch, val_acc, W1, stats)
#     filename = f'json/{hs}-{bs}-{lr}-{reg}-{num_epoch}.pickle'
#     savedb(obj,filename)
    
# save_general_pickle(hs, bs, lr, reg, num_epoch, val_acc, W1, stats)